In [1]:
#accessing protein sequences from ID
#code from ChatGPT

import requests

def get_protein_sequence(uniprot_id):
    # Construct the URL to retrieve the protein sequence data for the given ID
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"

    # Send a GET request to the URL and retrieve the response
    response = requests.get(url)

    # Extract the sequence data from the response text
    sequence = "".join(response.text.split("\n")[1:])

    return sequence

In [16]:
sequence = get_protein_sequence("B5ZC00")
print(sequence)

MKNKFKTQEELVNHLKTVGFVFANSEIYNGLANAWDYGPLGVLLKNNLKNLWWKEFVTKQKDVVGLDSAIILNPLVWKASGHLDNFSDPLIDCKNCKARYRADKLIESFDENIHIAENSSNEEFAKVLNDYEISCPTCKQFNWTEIRHFNLMFKTYQGVIEDAKNVVYLRPETAQGIFVNFKNVQRSMRLHLPFGIAQIGKSFRNEITPGNFIFRTREFEQMEIEFFLKEESAYDIFDKYLNQIENWLVSACGLSLNNLRKHEHPKEELSHYSKKTIDFEYNFLHGFSELYGIAYRTNYDLSVHMNLSKKDLTYFDEQTKEKYVPHVIEPSVGVERLLYAILTEATFIEKLENDDERILMDLKYDLAPYKIAVMPLVNKLKDKAEEIYGKILDLNISATFDNSGSIGKRYRRQDAIGTIYCLTIDFDSLDDQQDPSFTIRERNSMAQKRIKLSELPLYLNQKAHEDFQRQCQK


In [139]:
#reading in protein IDs
with open('test.txt', 'r') as text:
    ids = text.read()
    
#splitting IDs to make them parsable in for loop
ids = ids.split()

print(ids)

['A2Z669', 'B5ZC00', 'P07204', 'P20840']


In [106]:
#finding sequences that conform to motif N{P}[ST]{P}
#this function is generalizable to any protein motif that follows the same format of the above

def find_motif(seq, motif):
    
    #no easy way to check input lengths are the same so we are skipping that 
    
    checker = True #final return
    count = 0
    flag = 0 #used to check if a bracket is called, set to 1 with [] and 2 with {}
    hook = '' #sequence within brackets to check
    
    for i in motif:
        
        #this means the aa at this position can be anything 
        if i == '[' and flag == 0:
            flag = 1
            
        elif i == '{' and flag == 0:
            flag = 2

        elif i == ']' and flag == 1:
            flag = 0
            
            #checks if the aa is found in the hook sequence
            if seq[count] not in hook:
                checker = False
                
            hook = ''
            count +=1
            
        elif i == '}'and flag == 2:
            flag = 0
            
            #checks if the aa is NOT found in the hook sequence
            if seq[count] in hook:
                checker = False
    
            hook = ''
            count +=1
            
        elif flag == 0:
            
            #checks if aa matches motif at same position without other conditions
            if i != seq[count]:
                checker = False 
            count +=1
        
        elif flag == 1 or flag == 2:
            hook = hook + i
        
        else:
            print ('Error! Check motif')

    #if sequence is valid, return True, else return false 
    return checker, count
    

In [107]:
find_motif('NXSXSCT', 'N{P}[ST]{P}SCT')

(True, 7)

In [141]:
def mprt (ids, motif):
    
    #iterate over list of protein ids 
    for pid in ids:
        p_seq = get_protein_sequence(pid[0:6]) #pid 0:6 prevents miscalled ones from coming through
        check, motif_length = find_motif(p_seq, motif)
        loci = ''
        
        #needed to sanitize inputs...
        #print (p_seq)
        
        #iterate over all possible positions in protein sequence
        for i in range(0, len(p_seq) - motif_length+1):
            test_seq = p_seq[i:i+motif_length]
            
            #adds loci only if test_seq and motif match
            check, _ = find_motif(test_seq, motif)  # calling find_motif in if loop check did not work properly due to multiple outputs - executed as always True as a result
            if check:
                loci = loci + str(i + 1) + ' '
            
        #only print pid if there is a match
        if loci != '':
            print (pid)
            print (loci)
            
            

In [140]:
mprt(ids, 'N{P}[ST]{P}')

B5ZC00
85 118 142 306 395 
P07204
47 115 116 382 409 
P20840
79 109 135 248 306 348 364 402 485 501 614 


In [144]:
#reading in protein IDs
with open('rosalind.txt', 'r') as text:
    ids = text.read()
    
#splitting IDs to make them parsable in for loop
ids = ids.split()

print(ids)

['Q1JHI2', 'B4S2L7', 'P00304_ARA3_AMBEL', 'P81824_PABJ_BOTJA', 'P00744_PRTZ_BOVIN', 'Q3BRP8', 'P10643_CO7_HUMAN', 'Q67JS9', 'O08537_ESR2_MOUSE', 'Q5FTZ8', 'P02725_GLP_PIG', 'Q0IBF4', 'A9N9G8']


In [145]:
mprt(ids, 'N{P}[ST]{P}')

B4S2L7
33 
P00304_ARA3_AMBEL
41 
P81824_PABJ_BOTJA
25 
P00744_PRTZ_BOVIN
59 191 289 
P10643_CO7_HUMAN
202 754 
O08537_ESR2_MOUSE
17 55 407 
Q5FTZ8
49 62 
P02725_GLP_PIG
16 19 39 
Q0IBF4
136 
